# 초기 셋팅

In [ ]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.2 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292701 sha256=a381c7f17dbbb173beb44284c1d03d51582dbf778b50cc64a374f777b4e8e7ff
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, ht

In [ ]:
#glonnlp 임포트 오류 이걸로 해결
!pip install gluonnlp==0.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.9.1-cp310-cp310-linux_x86_64.whl size=557733 sha256=dd7bb242d75ceebd7691d3b1d84e10d1ca0684009e6ad387f02ae63852539ff9
  Stored in directory: /root/.cache/pip/wheels/fc/5b/9c/3295bb07f7c5544a96303a48988707816f44a536e8e1413922
Successfully built gluonnlp
  Attempting uninstall: gluonnlp
    Found existing installation: gluonnlp 0.8.0
    Uninstalling gluonnlp-0.8.0:
      Successfully uninstalled gluonnlp-0.8.0


In [ ]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-g9b1wj4b/kobert-tokenizer_6580bce9bf5d4896aef378003d8f232a
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-g9b1wj4b/kobert-tokenizer_6580bce9bf5d4896aef378003d8f232a
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=977229eccb8c061630006b29344b43c3d8650a91f2f73c65e280cee517a7b111
  Stored in directory: /tmp/pip-ephem-wheel-cache-qecqafyy/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# ★
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/비머/데이터/마애 상품 모음_이름 변경 후.csv', encoding = 'cp949')
df.rename(columns={'Unnamed: 0' : 'group'}, inplace=True)

df.head()

,group,prd_nm,review
0,0,센카 퍼펙트 휩 페이셜 워시,??세정력 별로에요\n디렉터파이님 영상 보면 선크림도 거의 못 지우고\n메이크업 후...
1,1,센카 퍼펙트 휩 페이셜 워시,콩만큼 짜도 거품이 와아악 나요\n향기도 청량하이 화장 지우는 맛 납니당… 헹구고 ...
2,2,센카 퍼펙트 휩 페이셜 워시,거품이 풍성해서 넘 좋아요 센카 브랜드는 워낙 클렌징폼으로 유명해서 의심없이 샀는데...
3,3,센카 퍼펙트 휩 페이셜 워시,올리브영 세일기간이라 저렴한 가격에 잘 산거 같아서 매우 만족합니다
4,4,센카 퍼펙트 휩 페이셜 워시,소량만으로도 쫀쫀한 거품이 아주 잘 나는 제품이에요\n몇년째 쓰고있는데 역시나 맘에듭니다


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24547 entries, 0 to 24546
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   group   24547 non-null  int64 
 1   prd_nm  24547 non-null  object
 2   review  24488 non-null  object
dtypes: int64(1), object(2)
memory usage: 575.4+ KB


In [ ]:
df['group'] = df['prd_nm'].map({'[증량] 마녀공장 퓨어&딥 클렌징 폼 1+1 기획 (120ml+120ml)' : 1,
                                '[단독기획] 비플레인 녹두 약산성 클렌징폼 (본품160ml+폼20ml+오일20ml 증정)' : 1,
                                '마녀공장 퓨어 클린징 오일': 1,
                                '바이오더마 센시비오': 1,
                                '닥터지 브라이트닝 필링젤' : 1,
                                '프리메라 페이셜 마일드 필링': 1,
                                '센카 퍼펙트 휩 페이셜 워시' :2, '메디필 레드 락토 콜라겐 클리어 300ml' :2,
                                '바닐라코 클린잇제로 클렌징밤 포어클래리파잉' :2, '메디필 엑스트라 슈퍼9 플러스 250ml (피지연화제)' :2,
                                '마녀공장 갈락 필링젤':2, '[단독대용량] 바닐라코 립 앤 아이 리무버 클리어 250ml':2,
                                '유리아쥬 이제악 클렌징 젤 150ml' :3, '[단독기획] 그라펜 제주해수 버블클렌저 150ml (단품 / 1+1 기획)':3,
                                '니베아 센서티브 미셀라 에어 클렌징 워터' :3, '에센허브 소이빈90 딥클렌징오일 오리지널':3,
                                '센카 올 클리어 시트 프레시 30매' :3, '궁중비책 마일드 선클렌징 티슈 (10매+10메+10매)':3,
                               '뷰 앰플 클렌저 기획세트(150ml+60ml)' :4, '[몬스타엑스 민혁 PICK]플로운 홀리바질 페이셜 팩투 클렌징폼 150ml' :4,
                                '유리프 키 페이스 클렌징 오일' :4, '바닐라코 클린잇제로 클렌징밤 너리싱 100ml':4,
                                '더말로지카 데일리 마이크로폴리언트' :4, '참존 징코 수딩 앤 클렌징 티슈 70매':4
                                })

1번 그룹 : 상위권 이성 소구인 제품

2번 그룹 : 상위권 감성 소구인 제품

3번 그룹 : 하위권 이성 소구인 제품

4번 그룹 : 하위권 감성 소구인 제품

In [ ]:
df['group'] = df['group'].map({3:1,
                               4:2,
                              1:1,
                              2:2})

In [ ]:
df['group'].value_counts()

1    13840
2    10707
Name: group, dtype: int64

In [ ]:
df.dropna(inplace = True)
df = df.reset_index().iloc[:,1:]

In [ ]:
new_data = df.copy()

In [ ]:
new_data.head()

,group,prd_nm,review
0,2,센카 퍼펙트 휩 페이셜 워시,??세정력 별로에요\n디렉터파이님 영상 보면 선크림도 거의 못 지우고\n메이크업 후...
1,2,센카 퍼펙트 휩 페이셜 워시,콩만큼 짜도 거품이 와아악 나요\n향기도 청량하이 화장 지우는 맛 납니당… 헹구고 ...
2,2,센카 퍼펙트 휩 페이셜 워시,거품이 풍성해서 넘 좋아요 센카 브랜드는 워낙 클렌징폼으로 유명해서 의심없이 샀는데...
3,2,센카 퍼펙트 휩 페이셜 워시,올리브영 세일기간이라 저렴한 가격에 잘 산거 같아서 매우 만족합니다
4,2,센카 퍼펙트 휩 페이셜 워시,소량만으로도 쫀쫀한 거품이 아주 잘 나는 제품이에요\n몇년째 쓰고있는데 역시나 맘에듭니다


In [ ]:
#범주형 데이터인 group을 숫자형 데이터로 변경
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(new_data['group'])
new_data['group'] = encoder.transform(new_data['group'])
new_data.head()

,group,prd_nm,review
0,1,센카 퍼펙트 휩 페이셜 워시,??세정력 별로에요\n디렉터파이님 영상 보면 선크림도 거의 못 지우고\n메이크업 후...
1,1,센카 퍼펙트 휩 페이셜 워시,콩만큼 짜도 거품이 와아악 나요\n향기도 청량하이 화장 지우는 맛 납니당… 헹구고 ...
2,1,센카 퍼펙트 휩 페이셜 워시,거품이 풍성해서 넘 좋아요 센카 브랜드는 워낙 클렌징폼으로 유명해서 의심없이 샀는데...
3,1,센카 퍼펙트 휩 페이셜 워시,올리브영 세일기간이라 저렴한 가격에 잘 산거 같아서 매우 만족합니다
4,1,센카 퍼펙트 휩 페이셜 워시,소량만으로도 쫀쫀한 거품이 아주 잘 나는 제품이에요\n몇년째 쓰고있는데 역시나 맘에듭니다


In [ ]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: 1, 1: 2}

In [ ]:
#review 데이터 str 데이터로 변경
data_list = []
for q, label in zip(new_data['review'], new_data['group']) :
  data = []
  data.append(q)
  data.append(str(label))

  data_list.append(data)

In [ ]:
len(data_list)

24488

In [ ]:
#train, test 데이터 분할
from sklearn.model_selection import train_test_split
train, test = train_test_split(data_list, test_size=0.2, random_state=42) # new_df 오탈자 수정
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 19590
test shape is: 4898


In [ ]:
# BERT 모델의 입력 데이터를 생성
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# 파라미터 조정

In [ ]:
# max_len 조정을 위한 eda
longest_sentence = max(df['review'], key=len)
avg_sentence_length = df['review'].apply(len).mean()

#print("가장 긴 문장:", longest_sentence)
print("문장의 길이:", len(longest_sentence))
print("문장 길이 평균:", avg_sentence_length)

문장의 길이: 1401
문장 길이 평균: 159.96377817706633


In [ ]:
## Setting parameters
max_len = 200 # 500 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 32
warmup_ratio = 0.1
num_epochs = 20
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
train[1]

['계속 쓰고있는 제품인데 자극없이 순하고 좋아요\n향도 인위적인 향도 아니고 은은하고 좋아요', '0']

In [ ]:
# bert에 내장된 기능으로 토큰화
tok = tokenizer.tokenize

data_train = BERTDataset(train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
# 데이터 형식 확인
data_train[0]

(array([   2, 2734, 7565, 6136, 5592,  517, 5330, 6359, 5400, 4129, 5330,
         742, 5579, 1119, 7848, 2421, 6857, 3888, 6881, 5439, 1100, 5703,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
#학습과 테스트에 필요한 미니배치 단위의 데이터를 로드
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# bert 기반 모델 생성
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# BERT 모델의 옵티마이저(optimizer)를 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
# BERT 모델의 옵티마이저(optimizer)와 스케줄러(scheduler)를 설정
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## 모델

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-36-e6a38b13095b>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/613 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.8462204337120056 train acc 0.28125
epoch 1 batch id 201 loss 0.6784666180610657 train acc 0.5219216417910447
epoch 1 batch id 401 loss 0.7796400785446167 train acc 0.5601620947630923
epoch 1 batch id 601 loss 0.5046029686927795 train acc 0.6135607321131448
epoch 1 train acc 0.6159597607395323


<ipython-input-36-e6a38b13095b>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/154 [00:00<?, ?it/s]

...

epoch 19 test acc 0.8299512987012987


  0%|          | 0/613 [00:00<?, ?it/s]

epoch 20 batch id 1 loss 0.002964641433209181 train acc 1.0
epoch 20 batch id 201 loss 0.0006127263186499476 train acc 0.9990671641791045
epoch 20 batch id 401 loss 0.39085254073143005 train acc 0.9985972568578554
epoch 20 batch id 601 loss 0.09607771039009094 train acc 0.9985960898502496
epoch 20 train acc 0.9985725938009788


  0%|          | 0/154 [00:00<?, ?it/s]

epoch 20 test acc 0.8297483766233766


# 평가

In [ ]:
model.eval()  # 모델을 평가 모드로 변경

total_samples = 0  # 전체 샘플 수
correct_predictions = 0  # 정확하게 예측한 샘플 수

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length = valid_length
    label = label.long().to(device)

    with torch.no_grad():
        out = model(token_ids, valid_length, segment_ids)
        _, predicted = torch.max(out, dim=1)  # 가장 높은 확률을 가진 클래스로 예측

    total_samples += label.size(0)  # 배치 내 샘플 수 추가
    correct_predictions += (predicted == label).sum().item()  # 정확한 예측 수 누적

accuracy = correct_predictions / total_samples  # 전체 정확도 계산
print("Accuracy: {:.2%}".format(accuracy))

Accuracy: 82.87%


# 문장 예측

In [ ]:
# 테스트 문장 예측
test_sentence = '이 제품은 저한테 잘 맞는거 같아요'
test_label = 1

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['review', 'group']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, vocab, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-47-247cd547fc58>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[ 4.3450, -4.1176]], device='cuda:0', grad_fn=<AddmmBackward0>)


In [ ]:
# 테스트 문장 예측
test_sentence = '세정력이 생각보다 괜찮아요'
test_label = 2

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['review', 'group']])
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, vocab, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  print(out)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
<ipython-input-49-6f26a4d72b88>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

tensor([[-3.0216,  2.8793]], device='cuda:0', grad_fn=<AddmmBackward0>)
